# Loading the libs

In [1]:
import nltk
from nltk.corpus import stopwords
import os
import numpy as np
import string
import sklearn
from sklearn import feature_extraction
import pandas as pd

/home/simon/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/simon/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/simon/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/simon/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## Downloading nltk corpus

In [20]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/simon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/simon/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
stopword = set(stopwords.words('english'))
punctuat = set(string.punctuation)

## Listing all files

In [5]:
pos_files = []
for root, dirs, files in os.walk("review_polarity/txt_sentoken/pos"):  
    for filename in files:
        if '.txt' in filename:
            pos_files.append(filename)
neg_files = []
for root, dirs, files in os.walk("review_polarity/txt_sentoken/neg"):  
    for filename in files:
        if '.txt' in filename:
            neg_files.append(filename)

## Reading the files

In [6]:
sentences = []
y = []
for filename in pos_files:
    with open('review_polarity/txt_sentoken/pos/'+filename, encoding='utf-8') as f:
        sentences.append(f.read().lower().translate(str.maketrans('','',string.punctuation)))
        y.append(1)
for filename in neg_files:
    with open('review_polarity/txt_sentoken/neg/'+filename, encoding='utf-8') as f:
        sentences.append(f.read().lower().translate(str.maketrans('','',string.punctuation)))
        y.append(0)      

In [7]:
filtered_sentence = [w for w in word_tokens if not w in stopword]
filtered_sentence = [w for w in word_tokens if not w in punctuat]

NameError: name 'word_tokens' is not defined

In [9]:
filterd = []
y = []
for sentence in pos_sentences:
    word_tokens = nltk.word_tokenize(sentence) 
    filtered_sentence = [w.lower() for w in word_tokens if not w in stopword]
    filtered_sentence = [w.lower() for w in word_tokens if not w in punctuat]
    filterd.append(filtered_sentence)
    y.append(1)
for sentence in neg_sentences:
    word_tokens = nltk.word_tokenize(sentence) 
    filtered_sentence = [w.lower() for w in word_tokens if not w in stopword]
    filtered_sentence = [w.lower() for w in word_tokens if not w in punctuat]
    filterd.append(filtered_sentence)
    y.append(0)

In [8]:
vectorizer = feature_extraction.text.TfidfVectorizer(stop_words = 'english')
X = vectorizer.fit_transform(sentences)

/home/simon/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1067: RuntimeWarning: overflow encountered in log
  idf = np.log(float(n_samples) / df) + 1.0
/home/simon/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1067: RuntimeWarning: invalid value encountered in log
  idf = np.log(float(n_samples) / df) + 1.0


In [9]:
X

<2000x47182 sparse matrix of type '<class 'numpy.float64'>'
	with 493343 stored elements in Compressed Sparse Row format>

In [10]:
contigency = pd.DataFrame(index=vectorizer.get_feature_names(),data = 0, columns=['pos', 'neg'])

In [11]:
contigency.head(20)

,pos,neg
00,0,0
000,0,0
0009f,0,0
000acre,0,0
000aweek,0,0
000foot,0,0
000paltry,0,0
007,0,0
007esque,0,0
00s,0,0


In [97]:
for word in contigency.index:
    for i in range(0, len(sentences)):
        if word in sentences[i] and i == 1:
            contigency.loc[word]['pos'] = contigency.loc[word]['pos'] + 1
        elif word in sentences[i] and i == 0:
            contigency.loc[word]['pos'] = contigency.loc[word]['neg'] + 1

In [100]:
contigency.head(20)

,pos,neg
00,0,0
000,0,0
0009f,0,0
000acre,0,0
000aweek,0,0
000foot,0,0
000paltry,0,0
007,0,0
007esque,0,0
00s,0,0


In [114]:
count_vectorizer_pos = feature_extraction.text.CountVectorizer()
count_vectorizer_neg = feature_extraction.text.CountVectorizer()

In [115]:
X_count_pos = count_vectorizer_pos.fit_transform(sentences[0:1000])
X_count_neg = count_vectorizer_neg.fit_transform(sentences[1000:2000])

In [116]:
for word in contigency.index:
    if word in count_vectorizer_pos.vocabulary_:
        contigency.loc[word]['pos'] = count_vectorizer_pos.vocabulary_[word]
    if word in count_vectorizer_neg.vocabulary_:
        contigency.loc[word]['neg'] = count_vectorizer_neg.vocabulary_[word]

Creating a matrix of size number of review * vocabulary size

For each word, check its presence in each text

In [17]:
word_presence = pd.DataFrame(index=vectorizer.get_feature_names(),data = 0, columns=np.arange(len(sentences)))

In [18]:
word_presence.head()

,0,1,2,3,4,5,6,7,8,9,...,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0009f,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
000acre,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
000aweek,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
for i in range(0, len(sentences)):
    word_tokens = nltk.word_tokenize(sentences[i]) 
    for word in word_presence.index:
        if word in word_tokens:
            word_presence.loc[word][i] = 1